<a href="https://colab.research.google.com/github/akanksha-ahuja/fcc-final-notebooks/blob/main/fcc_experiment_3_x_model_performance_ChebX_2_hyperparameters_k_3_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.manifold import TSNE
from sklearn.metrics import f1_score, matthews_corrcoef
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Performance of ChebX 2 layer variable size 

In [3]:
path_to_experiment_variable_depth_2_cheb_radius_k_3 = '/content/drive/MyDrive/FCC_Experiments_2021/model_chebnet/variable/depth_2/k_3/radius/'
path_to_experiment_variable_depth_2_cheb_knn_k_3 = '/content/drive/MyDrive/FCC_Experiments_2021/model_chebnet/variable/depth_2/k_3/knn/'
path_to_experiment_variable_depth_2_cheb_label_k_3 = '/content/drive/MyDrive/FCC_Experiments_2021/model_chebnet/variable/depth_2/k_3/label/'


In [4]:
path_to_experiment_variable_depth_2_cheb_radius_k_4 = '/content/drive/MyDrive/FCC_Experiments_2021/model_chebnet/variable/depth_2/k_4/radius/'

path_to_experiment_variable_depth_2_cheb_knn_k_4 = '/content/drive/MyDrive/FCC_Experiments_2021/model_chebnet/variable/depth_2/k_4/knn/'

path_to_experiment_variable_depth_2_cheb_label_k_4 = '/content/drive/MyDrive/FCC_Experiments_2021/model_chebnet/variable/depth_2/k_4/label/'


## Methods for performance metrics 
1. Event Accuracy 
2. Performance Metrics - F1 score, Matthew CorrCoef

## Event Accuracy: % of events classified with 100% accuracy 

In [5]:
# Caclulates the percentage of 100% correctly classified events
def calculate_event_accuracy(df):
    NUM_EVENTS = len(df.to_numpy().flatten())
    unique, counts = np.unique(df.to_numpy(), return_counts=True)
    acc_dict = dict(zip(unique, counts))
    # print(acc_dict)
    try: 
      num_events_classified_with_100_percent_acc = acc_dict[1.0]
      # print(num_events_classified_with_100_percent_acc)
      # print(NUM_EVENTS)
      event_accuracy = num_events_classified_with_100_percent_acc/NUM_EVENTS
      # print(event_accuracy)
      return event_accuracy
    except:
      print("There are no events with 100% percent accuracy")
      return 0 

# Saves the event accuracy over 3 datasets
def save_event_accuracy(event_accuracy_train, event_accuracy_valid, event_accuracy_test, path_to_experiment, file_name='df_event_accuracy'):
  event_accuracy  = [ event_accuracy_train, event_accuracy_valid, event_accuracy_test]
  df = pd.DataFrame([event_accuracy], columns=['event_accuracy_train', 'event_accuracy_valid', 'event_accuracy_test' ])
  df.to_csv(path_to_experiment + file_name, index=False)
  print(df)
  return df

# Calculates and stores the final df of event accuracy over train-valid-test
def store_event_accuracy(df_train_acc, df_valid_acc, df_test_acc, path_to_experiment):
  event_accuracy_train = calculate_event_accuracy(df_train_acc)
  event_accuracy_test = calculate_event_accuracy(df_valid_acc)
  event_accuracy_valid = calculate_event_accuracy(df_test_acc)
  df_event_accuracy = save_event_accuracy(event_accuracy_train, event_accuracy_valid, event_accuracy_test, path_to_experiment)
  return df_event_accuracy

## Methods of Performance metrics of node classification for each event

In [6]:
def calculate_f1_score(df_y, df_pred):
  f1_score_macro = f1_score(df_y, df_pred, average='macro')
  f1_score_micro = f1_score(df_y, df_pred, average='micro')
  f1_score_weighted = f1_score(df_y, df_pred, average='weighted')
  return f1_score_macro, f1_score_micro, f1_score_weighted

def calculate_matthews_correlation_coefficient(df_y, df_pred):
  matthews_correlation_coefficient = matthews_corrcoef(df_y, df_pred)
  return matthews_correlation_coefficient

# Saves both the mean and the list of performance metrics
def save_event_performance_metrics(f1_score_macro, f1_score_micro, f1_score_weighted, matthews_correlation_coefficient, path_to_experiment, file_name='df_performance_metrics'):
  performance_metrics  = [f1_score_macro, f1_score_micro, f1_score_weighted, matthews_correlation_coefficient]
  df = pd.DataFrame([performance_metrics], columns=['f1_score_macro', 'f1_score_micro', 'f1_score_weighted', 'matthews_corrcoef'])
  df.to_csv(path_to_experiment + file_name, index=False)
  print(df)
  return df

# Calculates the score of single event and then appends to a list
def overall_performance_metrics(df_y, df_pred):
  # Calculates the scores for all events in the test dataset 
  f1_score_macro_list, f1_score_micro_list, f1_score_weighted_list  = [], [], []
  matthews_correlation_coefficient_list = []
  
  for event_id in range(len(df_y)):
    f1_score_macro, f1_score_micro, f1_score_weighted = calculate_f1_score(df_y=df_y.iloc[event_id].dropna(), df_pred=df_pred.iloc[event_id].dropna())
    matthews_correlation_coefficient = calculate_matthews_correlation_coefficient(df_y=df_y.iloc[event_id].dropna(), df_pred=df_pred.iloc[event_id].dropna())
    f1_score_macro_list.append(f1_score_macro)
    f1_score_micro_list.append(f1_score_micro)
    f1_score_weighted_list.append(f1_score_weighted)
    matthews_correlation_coefficient_list.append(matthews_correlation_coefficient)

  return f1_score_macro_list, f1_score_micro_list, f1_score_weighted_list, matthews_correlation_coefficient_list

# Calculates the f1 score and matthew corrcoef score for all events 
def calculate_performance_metrics(df_y, df_pred, path_to_experiment):
  f1_score_macro_list, f1_score_micro_list, f1_score_weighted_list, matthews_correlation_coefficient_list = overall_performance_metrics(df_y, df_pred)
  df_performance_metrics = save_event_performance_metrics(f1_score_macro_list, f1_score_micro_list, f1_score_weighted_list, matthews_correlation_coefficient_list, path_to_experiment)
  return df_performance_metrics


# Calculates the mean of f1 score and matthew corrcoef score for all events 
def calculate_average_performance_metrics(df_performance_metrics, path_to_experiment):
  avg_f1_score_macro = sum(df_performance_metrics['f1_score_macro'][0])/len(df_performance_metrics['f1_score_macro'][0])
  avg_f1_score_micro = sum(df_performance_metrics['f1_score_micro'][0])/len(df_performance_metrics['f1_score_micro'][0])
  avg_f1_score_weighted = sum(df_performance_metrics['f1_score_weighted'][0])/len(df_performance_metrics['f1_score_weighted'][0])
  avg_matthews_correlation_coefficient = sum(df_performance_metrics['matthews_corrcoef'][0])/len(df_performance_metrics['matthews_corrcoef'][0])
  df_average_performance_metrics = save_event_performance_metrics(avg_f1_score_macro,avg_f1_score_micro,avg_f1_score_weighted,avg_matthews_correlation_coefficient, path_to_experiment, file_name='df_average_performance_metrics')
  return df_average_performance_metrics

**Compute the F1 score**, also known as balanced F-score or F-measure.

The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. The relative contribution of precision and recall to the F1 score are equal. The formula for the F1 score is:

```F1 = 2 * (precision * recall) / (precision + recall)```

In the multi-class and multi-label case, this is the average of the F1 score of each class with weighting depending on the average parameter.
- https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html


**The Matthews correlation coefficient** is used in machine learning as a measure of the quality of binary and multiclass classifications. It takes into account true and false positives and negatives and is generally regarded as a balanced measure which can be used even if the classes are of very different sizes. The MCC is in essence a correlation coefficient value between -1 and +1. A coefficient of +1 represents a perfect prediction, 0 an average random prediction and -1 an inverse prediction. The statistic is also known as the phi coefficient. Binary and multiclass labels are supported. 

- https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html

## Plot and Calculate average loss and average accuracies 

In [7]:
# Calculates average accuracy over a range of events for each epoch
def calculate_average_accuracy(df_acc):
  average_acc = []
  for event_id in range(len(df_acc)):
    average_acc_per_event = sum(df_acc.iloc[event_id].to_numpy())/len(df_acc.iloc[event_id].to_numpy())
    average_acc.append(average_acc_per_event)
  return average_acc

# Calculates average loss over a range of events for each epoch
def calculate_average_loss(df_loss):
  average_loss = []
  for event_id in range(len(df_loss)):
    average_loss_per_event = sum(df_loss.iloc[event_id].to_numpy())/len(df_loss.iloc[event_id].to_numpy())
    average_loss.append(average_loss_per_event)
  return average_loss

# Saves final accuracy for train-valid-test
def save_train_valid_test_accuracy_metrics(final_acc_train, final_acc_valid, final_test_acc, path_to_experiment, file_name='df_accuracy_metrics'):
  performance_metrics  = [final_acc_train, final_acc_valid, final_test_acc]
  df = pd.DataFrame([performance_metrics], columns=['final_acc_train', 'final_acc_valid', 'final_test_acc'])
  df.to_csv(path_to_experiment + file_name, index=False)
  print(df)
  return df

## Get metrics

In [8]:
def get_performance_metrics(df_y,  df_pred, df_train_acc,df_valid_acc, df_test_acc, path_to_experiment):
  df_performance_metrics = calculate_performance_metrics(df_y, df_pred, path_to_experiment)
  df_average_performance_metrics = calculate_average_performance_metrics(df_performance_metrics, path_to_experiment)
  df_event_accuracy = store_event_accuracy(df_train_acc, df_valid_acc, df_test_acc, path_to_experiment)
  return df_performance_metrics, df_average_performance_metrics, df_event_accuracy

def get_accuracy_metrics(df_train_acc, df_valid_acc, df_test_acc, df_train_loss, df_valid_loss, path_to_experiment):
  # List of average accuracy and losses over all events for train and valid datasets
  average_acc_train = calculate_average_accuracy(df_train_acc)
  average_acc_valid = calculate_average_accuracy(df_valid_acc)
  average_loss_train = calculate_average_loss(df_train_loss)
  average_loss_valid = calculate_average_loss(df_valid_loss)

  # List of event-wise accuracy on test dataset
  test_acc = df_test_acc.to_numpy().flatten()

  # Final single number - averaged across all events 
  final_test_acc = sum(test_acc)/len(test_acc)
  final_acc_train = sum(average_acc_train)/len(average_acc_train )
  final_acc_valid = sum(average_acc_valid)/len(average_acc_valid )

  # save the file for accuracy metrics
  df_accuracy_metrics = save_train_valid_test_accuracy_metrics(final_acc_train, final_acc_valid, final_test_acc, path_to_experiment)

  return  df_accuracy_metrics, average_acc_train,average_acc_valid, average_loss_train, average_loss_valid, test_acc, final_test_acc, final_acc_train 
    

## Get model outputs

In [9]:
def get_model_outputs(path_to_experiment, df_type = 'df_knn_'):
  # Get all files saved during model execution 
  df1 = pd.read_csv(path_to_experiment + df_type + "pred_mass_H_list_test").drop('Unnamed: 0', axis=1)
  df2 = pd.read_csv(path_to_experiment + df_type  + "pred_mass_H_list_train").drop('Unnamed: 0', axis=1)
  df3 = pd.read_csv(path_to_experiment + df_type + "pred_mass_H_list_valid").drop('Unnamed: 0', axis=1)
  df4 = pd.read_csv(path_to_experiment + df_type + "pred_mass_O_list_test").drop('Unnamed: 0', axis=1)
  df5 = pd.read_csv(path_to_experiment + df_type + "pred_mass_O_list_train").drop('Unnamed: 0', axis=1)
  df6 = pd.read_csv(path_to_experiment + df_type + "pred_mass_O_list_valid").drop('Unnamed: 0', axis=1)
  df7 = pd.read_csv(path_to_experiment + df_type + "pred_mass_Z_list_test").drop('Unnamed: 0', axis=1)
  df8 = pd.read_csv(path_to_experiment + df_type + "pred_mass_Z_list_train").drop('Unnamed: 0', axis=1)
  df9 = pd.read_csv(path_to_experiment + df_type + "pred_mass_Z_list_valid").drop('Unnamed: 0', axis=1)

  df10 = pd.read_csv(path_to_experiment + df_type + "test_acc.csv").drop('Unnamed: 0', axis=1)
  df11 = pd.read_csv(path_to_experiment + df_type + "test_pred.csv").drop('Unnamed: 0', axis=1)
  df12 = pd.read_csv(path_to_experiment + df_type + "test_y.csv").drop('Unnamed: 0', axis=1)
  df13 = pd.read_csv(path_to_experiment + df_type + "test_y_numpy.csv").drop('Unnamed: 0', axis=1)

  df14 = pd.read_csv(path_to_experiment + df_type + "train_acc.csv").drop('Unnamed: 0', axis=1)
  df15 = pd.read_csv(path_to_experiment + df_type + "train_loss.csv").drop('Unnamed: 0', axis=1)
  df16 = pd.read_csv(path_to_experiment + df_type + "train_pred.csv").drop('Unnamed: 0', axis=1)
  df17 = pd.read_csv(path_to_experiment + df_type + "train_time_per_epoch_list.csv").drop('Unnamed: 0', axis=1)
  df18 = pd.read_csv(path_to_experiment + df_type + "train_y.csv").drop('Unnamed: 0', axis=1)
  df19 = pd.read_csv(path_to_experiment + df_type + "train_y_numpy.csv").drop('Unnamed: 0', axis=1)

  df20 = pd.read_csv(path_to_experiment + df_type + "valid_acc.csv").drop('Unnamed: 0', axis=1)
  df21 = pd.read_csv(path_to_experiment + df_type + "valid_loss.csv").drop('Unnamed: 0', axis=1)
  df22 = pd.read_csv(path_to_experiment + df_type + "valid_pred.csv").drop('Unnamed: 0', axis=1)
  df23 = pd.read_csv(path_to_experiment + df_type + "valid_time_per_epoch_list.csv").drop('Unnamed: 0', axis=1)
  df24 = pd.read_csv(path_to_experiment + df_type + "valid_y.csv").drop('Unnamed: 0', axis=1)
  df25 = pd.read_csv(path_to_experiment + df_type + "valid_y_numpy.csv").drop('Unnamed: 0', axis=1)

  df26 = pd.read_csv(path_to_experiment + df_type + "true_mass_H_list_test").drop('Unnamed: 0', axis=1)
  df27 = pd.read_csv(path_to_experiment + df_type + "true_mass_H_list_train").drop('Unnamed: 0', axis=1)
  df28 = pd.read_csv(path_to_experiment + df_type + "true_mass_H_list_valid").drop('Unnamed: 0', axis=1)
  df29 = pd.read_csv(path_to_experiment + df_type + "true_mass_O_list_test").drop('Unnamed: 0', axis=1)
  df30 = pd.read_csv(path_to_experiment + df_type + "true_mass_O_list_train").drop('Unnamed: 0', axis=1)
  df31 = pd.read_csv(path_to_experiment + df_type + "true_mass_O_list_valid").drop('Unnamed: 0', axis=1)
  df32 = pd.read_csv(path_to_experiment + df_type + "true_mass_Z_list_test").drop('Unnamed: 0', axis=1)
  df33 = pd.read_csv(path_to_experiment + df_type + "true_mass_Z_list_train").drop('Unnamed: 0', axis=1)
  df34 = pd.read_csv(path_to_experiment + df_type + "true_mass_Z_list_valid").drop('Unnamed: 0', axis=1)

  # Fixed these two 
  df35 = pd.read_csv(path_to_experiment + df_type + "train_pred_numpy.csv").drop('Unnamed: 0', axis=1)
  df36 = pd.read_csv(path_to_experiment + df_type + "valid_pred_numpy.csv").drop('Unnamed: 0', axis=1)
  
  return df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17, df18, df19, df20, df21, df22, df23, df24, df25, df26, df27, df28, df29, df30, df31, df32, df33, df34, df35, df36

# Performance metrics: ChebNet 2 layer variable for KNN where cheb_k = 3

In [10]:
# Path to experiment KNN
path_to_experiment_variable_depth_2_cheb_knn_k_3 = '/content/drive/MyDrive/FCC_Experiments_2021/model_chebnet/variable/depth_2/k_3/knn/'


In [11]:
# Get dataframes for metrics
df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17, df18, df19, df20, df21, df22, df23, df24, df25, df26, df27, df28, df29, df30, df31, df32, df33, df34, df35, df36 =  get_model_outputs(path_to_experiment=path_to_experiment_variable_depth_2_cheb_knn_k_3, df_type = 'df_knn_')

In [12]:
# Calculate metrics
df_performance_metrics, df_average_performance_metrics, df_event_accuracy = get_performance_metrics(df_y=df13,  df_pred= df11, df_train_acc=df14,df_valid_acc=df20, df_test_acc=df10, path_to_experiment=path_to_experiment_variable_depth_2_cheb_knn_k_3)
df_accuracy_metrics, average_acc_train,average_acc_valid, average_loss_train, average_loss_valid, test_acc, final_test_acc, final_acc_train  = get_accuracy_metrics(df_train_acc= df14, df_valid_acc=df20, df_test_acc=df10, df_train_loss=df15, df_valid_loss=df21, path_to_experiment=path_to_experiment_variable_depth_2_cheb_knn_k_3)

                                      f1_score_macro  ...                                  matthews_corrcoef
0  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...  ...  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...

[1 rows x 4 columns]
   f1_score_macro  f1_score_micro  f1_score_weighted  matthews_corrcoef
0        0.998834        0.999753           0.999776           0.997875
   event_accuracy_train  event_accuracy_valid  event_accuracy_test
0              0.925444                 0.989             0.923353
   final_acc_train  final_acc_valid  final_test_acc
0         0.997944         0.998132        0.999753


# Performance metrics: ChebNet 2 layer variable for Radius for cheb_k=3


In [13]:
# Path to experiment for radius
path_to_experiment_variable_depth_2_cheb_radius_k_3 = '/content/drive/MyDrive/FCC_Experiments_2021/model_chebnet/variable/depth_2/k_3/radius/'


In [14]:
#  Get dataframes for metrics
df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17, df18, df19, df20, df21, df22, df23, df24, df25, df26, df27, df28, df29, df30, df31, df32, df33, df34, df35, df36 =  get_model_outputs(path_to_experiment=path_to_experiment_variable_depth_2_cheb_radius_k_3, df_type = 'df_radius_')

In [15]:
# Calculate metrics
df_performance_metrics, df_average_performance_metrics, df_event_accuracy = get_performance_metrics(df_y=df13,  df_pred= df11, df_train_acc=df14,df_valid_acc=df20, df_test_acc=df10, path_to_experiment=path_to_experiment_variable_depth_2_cheb_radius_k_3)
df_accuracy_metrics, average_acc_train,average_acc_valid, average_loss_train, average_loss_valid, test_acc, final_test_acc, final_acc_train  = get_accuracy_metrics(df_train_acc= df14, df_valid_acc=df20, df_test_acc=df10, df_train_loss=df15, df_valid_loss=df21, path_to_experiment=path_to_experiment_variable_depth_2_cheb_radius_k_3)

                                      f1_score_macro  ...                                  matthews_corrcoef
0  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...  ...  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...

[1 rows x 4 columns]
   f1_score_macro  f1_score_micro  f1_score_weighted  matthews_corrcoef
0        0.991959        0.998707           0.998685           0.985699
   event_accuracy_train  event_accuracy_valid  event_accuracy_test
0              0.749824                 0.941             0.756627
   final_acc_train  final_acc_valid  final_test_acc
0         0.993505         0.993939        0.998707


# Performance metrics: ChebNet 2 layer variable for Label for cheb_k=3

In [16]:
# Path to experiment for label
path_to_experiment_variable_depth_2_cheb_label_k_3 = '/content/drive/MyDrive/FCC_Experiments_2021/model_chebnet/variable/depth_2/k_3/label/'


In [17]:
#  Get dataframes for metrics
df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17, df18, df19, df20, df21, df22, df23, df24, df25, df26, df27, df28, df29, df30, df31, df32, df33, df34, df35, df36 =  get_model_outputs(path_to_experiment=path_to_experiment_variable_depth_2_cheb_label_k_3, df_type = 'df_label_')

In [18]:
# Calculate metrics
df_performance_metrics, df_average_performance_metrics, df_event_accuracy = get_performance_metrics(df_y=df13,  df_pred= df11, df_train_acc=df14,df_valid_acc=df20, df_test_acc=df10, path_to_experiment=path_to_experiment_variable_depth_2_cheb_label_k_3)
df_accuracy_metrics, average_acc_train,average_acc_valid, average_loss_train, average_loss_valid, test_acc, final_test_acc, final_acc_train  = get_accuracy_metrics(df_train_acc= df14, df_valid_acc=df20, df_test_acc=df10, df_train_loss=df15, df_valid_loss=df21, path_to_experiment=path_to_experiment_variable_depth_2_cheb_label_k_3)

                                      f1_score_macro  ...                                  matthews_corrcoef
0  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...  ...  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...

[1 rows x 4 columns]
   f1_score_macro  f1_score_micro  f1_score_weighted  matthews_corrcoef
0             1.0             1.0                1.0                1.0
   event_accuracy_train  event_accuracy_valid  event_accuracy_test
0              0.996791                   1.0             0.997578
   final_acc_train  final_acc_valid  final_test_acc
0         0.999459         0.999699             1.0


# Performance metrics: ChebNet 2 layer variable for KNN where cheb_k = 4

In [19]:
# Path to experiment KNN
path_to_experiment_variable_depth_2_cheb_knn_k_4 = '/content/drive/MyDrive/FCC_Experiments_2021/model_chebnet/variable/depth_2/k_4/knn/'

# Get dataframes for metrics
df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17, df18, df19, df20, df21, df22, df23, df24, df25, df26, df27, df28, df29, df30, df31, df32, df33, df34, df35, df36 =  get_model_outputs(path_to_experiment=path_to_experiment_variable_depth_2_cheb_knn_k_4, df_type = 'df_knn_')
# Calculate metrics
df_performance_metrics, df_average_performance_metrics, df_event_accuracy = get_performance_metrics(df_y=df13,  df_pred= df11, df_train_acc=df14,df_valid_acc=df20, df_test_acc=df10, path_to_experiment=path_to_experiment_variable_depth_2_cheb_knn_k_4)
df_accuracy_metrics, average_acc_train,average_acc_valid, average_loss_train, average_loss_valid, test_acc, final_test_acc, final_acc_train  = get_accuracy_metrics(df_train_acc= df14, df_valid_acc=df20, df_test_acc=df10, df_train_loss=df15, df_valid_loss=df21, path_to_experiment=path_to_experiment_variable_depth_2_cheb_knn_k_4)

                                      f1_score_macro  ...                                  matthews_corrcoef
0  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...  ...  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...

[1 rows x 4 columns]
   f1_score_macro  f1_score_micro  f1_score_weighted  matthews_corrcoef
0         0.99841        0.999661           0.999692           0.997101
   event_accuracy_train  event_accuracy_valid  event_accuracy_test
0              0.916219                 0.985             0.917618
   final_acc_train  final_acc_valid  final_test_acc
0         0.997686         0.997952        0.999661


# Performance metrics: ChebNet 2 layer variable for radius where cheb_k = 4

In [20]:
# Path to experiment for radius
path_to_experiment_variable_depth_2_cheb_knn_k_4 = '/content/drive/MyDrive/FCC_Experiments_2021/model_chebnet/variable/depth_2/k_4/knn/'

#  Get dataframes for metrics
df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17, df18, df19, df20, df21, df22, df23, df24, df25, df26, df27, df28, df29, df30, df31, df32, df33, df34, df35, df36 =  get_model_outputs(path_to_experiment=path_to_experiment_variable_depth_2_cheb_radius_k_4, df_type = 'df_radius_')
# Calculate metrics
df_performance_metrics, df_average_performance_metrics, df_event_accuracy = get_performance_metrics(df_y=df13,  df_pred= df11, df_train_acc=df14,df_valid_acc=df20, df_test_acc=df10, path_to_experiment=path_to_experiment_variable_depth_2_cheb_radius_k_4)
df_accuracy_metrics, average_acc_train,average_acc_valid, average_loss_train, average_loss_valid, test_acc, final_test_acc, final_acc_train  = get_accuracy_metrics(df_train_acc= df14, df_valid_acc=df20, df_test_acc=df10, df_train_loss=df15, df_valid_loss=df21, path_to_experiment=path_to_experiment_variable_depth_2_cheb_radius_k_4)

                                      f1_score_macro  ...                                  matthews_corrcoef
0  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...  ...  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...

[1 rows x 4 columns]
   f1_score_macro  f1_score_micro  f1_score_weighted  matthews_corrcoef
0        0.978061        0.997114           0.996884           0.959128
   event_accuracy_train  event_accuracy_valid  event_accuracy_test
0              0.713127                 0.883             0.714971
   final_acc_train  final_acc_valid  final_test_acc
0         0.992542         0.992848        0.997114


# Performance metrics: ChebNet 2 layer variable for Label where cheb_k = 4

In [21]:
# Path to experiment for label
path_to_experiment_variable_depth_2_cheb_label_k_4 = '/content/drive/MyDrive/FCC_Experiments_2021/model_chebnet/variable/depth_2/k_4/label/'

#  Get dataframes for metrics
df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17, df18, df19, df20, df21, df22, df23, df24, df25, df26, df27, df28, df29, df30, df31, df32, df33, df34, df35, df36 =  get_model_outputs(path_to_experiment=path_to_experiment_variable_depth_2_cheb_label_k_4, df_type = 'df_label_')
# Calculate metrics
df_performance_metrics, df_average_performance_metrics, df_event_accuracy = get_performance_metrics(df_y=df13,  df_pred= df11, df_train_acc=df14,df_valid_acc=df20, df_test_acc=df10, path_to_experiment=path_to_experiment_variable_depth_2_cheb_label_k_4)
df_accuracy_metrics, average_acc_train,average_acc_valid, average_loss_train, average_loss_valid, test_acc, final_test_acc, final_acc_train  = get_accuracy_metrics(df_train_acc= df14, df_valid_acc=df20, df_test_acc=df10, df_train_loss=df15, df_valid_loss=df21, path_to_experiment=path_to_experiment_variable_depth_2_cheb_label_k_4)

                                      f1_score_macro  ...                                  matthews_corrcoef
0  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...  ...  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...

[1 rows x 4 columns]
   f1_score_macro  f1_score_micro  f1_score_weighted  matthews_corrcoef
0             1.0             1.0                1.0                1.0
   event_accuracy_train  event_accuracy_valid  event_accuracy_test
0               0.99752                   1.0             0.998451
   final_acc_train  final_acc_valid  final_test_acc
0         0.999445         0.999706             1.0
